# Intro
In this file, we will do a classification problem using the dataset CIFAR10, the dataset contains 50K images for the training set from 10 different classes(ie 5K for each class), and 10K for the test set(ie 1K for each class). One image of CIFAR10 is of size 3x32x32. The different classes in CIFAR10 are 

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

The test set contains images. Our model will be a Convolutional Neural Network
So these are the next steps for our training:
    
    1- We do the data preprocessing (loading, normalizing, data augmentation ...)
    2- We define our model (Convolutional Neural Network)
    3- We define our loss function
    4- Train the model on the training data
    5- Test the model on the test data
    6- Print the accuracy on each class of the dataset

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import torchvision
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from torch import optim
from torchvision.datasets import CIFAR10
from torchvision import transforms
random.seed(23)

# 1- Load and Normalize the data - Data Augmentation

In [ ]:
#visualize some data first in CIFAR10
data = CIFAR10('./cifar10', download= False, train= True)

#Function for displaying images
%matplotlib inline
mpl.rcParams['figure.figsize'] = 20, 40

#img = torch.stack( (torch.stack((torch.asarray(dataset[i][0]) for _ in range(n)),dim = 0)
#                    for i in range(10)), dim = 1)
    
def show_dataset(dataset, n = 4):
    img = np.vstack((np.hstack((np.asarray(dataset[i][0]) for _ in range(n)))
                    for i in range(10)))
    plt.imshow(img)
    plt.axis('off')

show_dataset(data, 3)

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),    
    ])

trainset = CIFAR10('./cifar10', download = False, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size= 25, shuffle= True)
testset = CIFAR10('./cifar10', download = False, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size= 25, shuffle=True)

In [ ]:
print("Training set: ",(trainset))
print("Test set: ", (testset))

# 2- Define our CNN

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=9, kernel_size=5)  # output size after conv = 28, after pool= 14
        self.conv2 = nn.Conv2d(in_channels=9, out_channels=18, kernel_size=4) # output size after conv = 10, after pool= 5
        self.fc1 = nn.Linear(in_features=18*5*5, out_features=1000)
        self.fc2 = nn.Linear(in_features=1000, out_features=500)
        self.fc3 = nn.Linear(in_features=500, out_features=10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        x = x.view(-1, 18*5*5)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
        
model = MyNet()

In [ ]:
for name, param in model.named_parameters():
    print(name, param.size())

# 3- Define our Loss Function

In [ ]:
# Since we are dealing with a classification problem, we define a cross entropy loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)
len(trainset)

# 4- Train our model

In [ ]:
lossList = []
do = nn.Dropout(p = 0.5) #dropout to avoid overfitting

for epoch in range(25):
    correct = 0
    for data in trainloader:
        inp, labels = data   #get the inputs of the training set        
        
        optimizer.zero_grad()   #set the optimizer to zero
        
        inputs = do(inp)   #apply the dropout
        outputs = model(inputs) #forward pass
        
        loss = criterion(outputs, labels) #compute the loss
        loss.backward()      #backward pass
        optimizer.step()     #updating parameters
        
        pred = torch.argmax(outputs, dim= 1)
        for j in range(len(inputs)):
            if (pred[j]==labels[j]):
                correct += 1
    print("Epoch:", epoch+1," - Loss=", loss.item(), "- Training Accuracy=", 100*correct / len(trainset))
    lossList.append(loss.item())
        
print("Finished Training")

#plot the loss function
epoch_range = range(0, len(lossList))
plt.figure(figsize=[7,5])
plt.plot(epoch_range, lossList)
plt.show()

# 5- Test our model

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        #get the inputs of the test set
        inputs, labels = data
        
        outputs = model(inputs)
        pred = torch.argmax(outputs, dim=1) #return the index which is the number of the corresponding class
        for i in range (len(pred)):
            if (pred[i].item()==labels[i].item()):
                correct += 1
        total += len(labels)
        #print("Accuracy =", correct / total)
print("Accuracy over 10000 images =", 100*correct / total,"%")